In [93]:
from bs4 import BeautifulSoup
import requests
import json

import time
import numpy as np
import pandas as pd 

import re

from tqdm import tqdm
import unicodedata

## Demagog

In [2]:
demagog_statements_url = 'https://demagog.org.pl/wypowiedzi/page/'
page=requests.get(f'{demagog_statements_url}1')

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
n_pages = int(soup.findAll("a", class_="page-numbers")[1].text)
n_pages

83

In [5]:
txt_list = []
assestment_list = []
author_list = []
source_list = []

for i in range(1,n_pages+1):
    
    page=requests.get(f'{demagog_statements_url}{i}')
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for s in tqdm(soup.findAll("h2", class_="mt-0 mb-1 title-archive"), 
                  desc=f'page {i} of {n_pages}', 
                  position=0
                 ):
        statement_url = s.find('a').get("href")

        page_statement = requests.get(statement_url)
        soup_statement = BeautifulSoup(page_statement.content, 'html.parser')

        txt = unicodedata.normalize(soup_statement.find('blockquote', class_='hyphenate target-blank twitter-tweet') \
                .find('p').text)

        assestment = unicodedata.normalize(soup_statement.find_all('p', {"class": "ocena"})[0].text)
        author = unicodedata.normalize("NFKD", 
                                       soup_statement.find('div', class_='h2 person-name mt-0 count-text').find('a').text)
        source = unicodedata.normalize("NFKD", 
                                       soup_statement.find('div', class_='w-100 text-right date-content target-blank').find('p').text)
        
        txt_list.append(txt)
        assestment_list.append(assestment)
        author_list.append(author)
        source_list.append(source)
        
        time.sleep(1+np.random.randint(5)/5)

page 1 of 83:   0%|                                      | 0/60 [00:03<?, ?it/s]


In [19]:
unicodedata.normalize("NFKD", 
soup_statement.find('div', class_='w-100 text-right date-content target-blank').find('p').text
                     )

'Poranna rozmowa w RMF FM, 18.05.2022'

In [16]:
'Poranna rozmowa w\xa0RMF FM, 18.05.2022'.decode("string_escape")

LookupError: unknown encoding: string_escape

In [72]:
df_out = pd.DataFrame({'assestment' : assestment_list, 'text' : txt_list})
df_out.to_csv('../datasets/demagog.csv', sep=';', index=False)

## Oko.press

In [82]:
# https://stackoverflow.com/questions/48477688/scrape-page-with-load-more-results-button

In [81]:
oko_press_statements_url = 'https://oko.press/kategoria/prawda-czy-falsz/strona/'
page=requests.get(f'{oko_press_statements_url}{1}')

In [78]:
soup = BeautifulSoup(page.content, 'html.parser')

In [79]:
soup.find_all('a', {'class' : 'text__quotation'})

[<a class="text__quotation" href="https://oko.press/wypowiedzi/morawiecki-konstytucja-3-maja-wykluczala-niewole/" title="Mateusz Morawiecki">
 <p>Zresztą, polska Konstytucja była nawet dalej idąca niż ta zza oceanu, bo wykluczała niewolnictwo.</p>
 </a>,
 <a class="text__quotation" href="https://oko.press/wypowiedzi/morawiecki-o-ruskim-gazie/" title="Mateusz Morawiecki">
 <p>Od dzisiaj, kiedy Rosja przestała przesyłać swój gaz do Polski, przestaliśmy w jakikolwiek sposób od Rosji zależeć.</p>
 </a>,
 <a class="text__quotation" href="https://oko.press/wypowiedzi/ilu-mamy-uchodzcow-w-polsce/" title="Michał Dworczyk">
 <p>Możemy powiedzieć, że mamy między 1,5 a 2 mln wojennych uchodźców w Polsce</p>
 </a>]

## Fakehunter (only linkt to sources)

In [86]:
fakehunter_statements_url = f'https://panel-api.fakehunter.pap.pl/news/published/news?page={j}'
response = requests.get(fakehunter_statements_url)

In [94]:
fake_hunter = []
for j in tqdm(range(1, 124)):
    fakehunter_statements_url = f'https://panel-api.fakehunter.pap.pl/news/published/news?page={j}'
    response = requests.get(fakehunter_statements_url)

    response_dict = response.json()
    
    for r in response_dict['results']:
        verdict = r['expert_opinion']['verdict']
        title = r['title']
        url = r['url']
        
        fake_hunter.append([verdict, title, url])

100%|█████████████████████████████████████████| 123/123 [01:17<00:00,  1.59it/s]


In [96]:
len(fake_hunter)

2459

In [100]:
df_fh = pd.DataFrame(fake_hunter, columns=['verdict', 'title', 'url'])
df_fh.shape

(2459, 3)

In [103]:
df_fh = df_fh[df_fh['url'].str.contains('twitter.com')]
df_fh.shape

(699, 3)

In [104]:
df_fh.to_csv('../datasets/scrapped/fake_hunter.csv')

## fakenews.pl